In [1]:
import os
from tqdm import tqdm

import config
import modules.dataloaders as detection_data_loader
import modules.metrics as metrics

import numpy as np
import torch
import torchmetrics
from torchmetrics.detection.mean_ap import MeanAveragePrecision

INFO:albumentations.check_version:A new version of Albumentations is available: 1.4.14 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
import onnx
import onnxruntime

# Check Models

In [3]:
bed_manual_optim_name = './onnx_models/bed_manual_optim_detection_epoch=144_cpu.onnx'
bed_manual_optim_model = onnx.load(bed_manual_optim_name)
onnx.checker.check_model(bed_manual_optim_model)

# Helper function to convert pytorch tensors to numpy. Useful to handle datatset output.

In [4]:
def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# Detection Loaders

In [5]:
detection_loader = detection_data_loader.get_dfire_val_loader()


TEST DFire dataset
DFire Removed wrong images: 0
DFire Removed due to overlapping: 310
DFire Removed due to more than 10: 13

Test dataset len: 3983


# Evaluate Detector

In [6]:
map_metric = MeanAveragePrecision(
    box_format='xyxy',
    iou_thresholds=[config.IOU_THRESHOLD],
    class_metrics=True, # Enables separated metrics for each class
    #average='micro',
    extended_summary=False).to('cpu')

### Evaluation Function

In [7]:
def eval_detector_onnx(loader, model_name, score_thres):

    ort_session = onnxruntime.InferenceSession(model_name, providers=["CPUExecutionProvider"])

    map_metric.reset()
    
    loop = tqdm(loader, desc='Validating', leave=True)

    for batch_idx, (img, label) in enumerate(loop):

        for idx in range(config.BATCH_SIZE):
            
            ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img[idx].unsqueeze(dim=0))}
            out = ort_session.run(None, ort_inputs)
            
            # out of onnx session is a list: [out_tensor with batch dim] -> [ (1,12,7,7) ] 
            # -> out[0] = (1,12,7,7)
            # -> out[0][0] = (12,7,7)
            #print(f'Out type: {type(out[0])} - Output shape: {out[0].shape}')
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)
            #print(f'Out shape after permute: {out.shape}')
            #print(f'Out shape after indexing: {out[0].shape}')
            
            # Label should be [xc, yc, w, h, score=1, smoke, fire] in 7x7 square
            #print(f'Label indexed shape: {label[idx].shape}')
            
        # Mean Average Precision
            target_boxes = metrics.get_true_boxes(label[idx].detach().to('cpu'))
            pred_boxes = metrics.get_pred_boxes(
                model_out = out[0].detach().to('cpu'),
                score_threshold=score_thres)
            map_metric.update(preds = pred_boxes, target = target_boxes)
    
    meanAP = map_metric.compute()
    map_metric.reset()

    print(f'Smoke -> AP: {meanAP["map_per_class"][0].item():.4f} - AR: {meanAP["mar_100_per_class"][0].item():.4f}')
    print(f'Fire -> AP: {meanAP["map_per_class"][1].item():.4f} - AR: {meanAP["mar_100_per_class"][1].item():.4f}')
    print(f'mAP: {meanAP["map_50"].item():.4f}')
    
    return (
        {'mAP': meanAP['map_50'].item(),
         'AP': [meanAP['map_per_class'][0].item(), meanAP['map_per_class'][1].item()],
         'AR': [meanAP['mar_100_per_class'][0].item(), meanAP['mar_100_per_class'][1].item()]
        }
    )

In [8]:
print("\n________________________________ BED manual optim _______________________________")
_ = eval_detector_onnx(
    detection_loader, 
    bed_manual_optim_name,
    0.2
)


________________________________ BED manual optim _______________________________


Validating: 100%|██████████████████████████████████████████████████████████████████████████████████| 62/62 [02:26<00:00,  2.36s/it]


Smoke -> AP: 0.4611 - AR: 0.5522
Fire -> AP: 0.2658 - AR: 0.3712
mAP: 0.3635


# Aladdin Metrics to Get Precision and Precision-Recall Curves

In [9]:
import modules.metrics_mAP_aladdin as aladdin_metrics

In [10]:
''' ============================
    Cell to Box Mask
============================ '''
cell2box_mask = torch.zeros((config.S, config.S, 2))
for i in range(config.S):
    for j in range(config.S):
        cell2box_mask[i,j,0] = j
        cell2box_mask[i,j,1] = i  

### Aladdin mAP modified for ONNX detection with thresholds

In [11]:
def aladdin_get_bboxes(
    loader,
    detector_model_name,
    S=config.S,
    B=config.B,
    C=config.C,
    mask=cell2box_mask,
    device='cpu',
    iou_threshold=0.5,
    score_thres=0.2,
    box_format="midpoint"):
    
    '''
    
    Return:
        - all_pred_boxes
        - all_true_boxes
        Format: [train_idx, class_prediction, prob_score, x1, y1, x2, y2]
    '''
    
    detect_session = onnxruntime.InferenceSession(detector_model_name, providers=["CPUExecutionProvider"])
    
    
    all_pred_boxes = []
    all_true_boxes = []

    # Original Code
    # make sure model is in eval before get bboxes
    #model.eval()
    train_idx = 0

    loop = tqdm(loader, desc='Get Boxes', leave=True)
    for batch_idx, (imgs, labels) in enumerate(loop):
        # Original Code
        # Inference in GPU. Move tensor to CPU in outcell_2_outboxes
        # imgs = imgs.to(device)
        # labels = labels.to(device)


        for idx in range(config.BATCH_SIZE):
            
            detect_inputs = {detect_session.get_inputs()[0].name: to_numpy(imgs[idx].unsqueeze(dim=0))}
            out = detect_session.run(None, detect_inputs)
            out = torch.tensor(np.array(out[0]))
            out = out.permute(0, 2, 3, 1)         
            
            # Original Code
            # with torch.no_grad():
            #     predictions = model(imgs)

            # Original Code
            # Remove Permute from the model
            #predictions = predictions.permute(0, 2, 3, 1) # Original Code

            #batch_size = imgs.shape[0] # Original Code

            true_bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=labels[idx].unsqueeze(dim=0), 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=False)
            bboxes = aladdin_metrics.outcell_2_outboxes(
                out_cells=out, 
                S=S, B=B, C=C, 
                mask=mask, 
                device='cpu', # Changed to cpu
                is_pred=True)

            for idx in range(1): # Only 1 image every time, due to ONNX prediction in CPU
                nms_boxes = aladdin_metrics.nms_yv1_getBBoxes(
                    bboxes[idx],
                    iou_threshold=iou_threshold,
                    threshold=score_thres,
                    box_format=box_format, # Midpoint, to use iou_tensor inside
                )


                # Plot some examples
                #if batch_idx == 0 and idx == 0:
                #    plot_image(x[idx].permute(1,2,0).to("cpu"), nms_boxes)
                #    print(nms_boxes)

                for nms_box in nms_boxes:
                    all_pred_boxes.append([train_idx] + nms_box)

                for box in true_bboxes[idx]:
                    # many will get converted to 0 pred, as bboxes have Conf = 1 and the rest are 0
                    if box[1] > score_thres:
                        all_true_boxes.append([train_idx] + box)

                train_idx += 1

    #model.train()
    return all_pred_boxes, all_true_boxes

# Function to Print mAP metrics

In [12]:
def print_metrics(mAP_metrics):
    mAP, avg_prec, cls_prec, cls_rec = mAP_metrics
    
    mAP_str = "mAP @0.50"
    smoke = "Smoke"
    fire = "Fire"
    
    print(f'{mAP_str:<12}' + f'{mAP:.4f}')
    print('Average Precision')
    print(f'- {smoke:<10}' + f'{avg_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{avg_prec[1]:.4f}')
    print('Class Precision')
    print(f'- {smoke:<10}' + f'{cls_prec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_prec[1]:.4f}')  
    print('Class Recall')
    print(f'- {smoke:<10}' + f'{cls_rec[0]:.4f}')
    print(f'- {fire:<10}' + f'{cls_rec[1]:.4f}')
    print('Class F1-Score')
    smoke_f1 = 2 * (cls_prec[0] * cls_rec[0]) / (cls_prec[0] + cls_rec[0])
    fire_f1 = 2 * (cls_prec[1] * cls_rec[1]) / (cls_prec[1] + cls_rec[1])
    print(f'- {smoke:<10}' + f'{smoke_f1:.4f}')
    print(f'- {fire:<10}' + f'{fire_f1:.4f}')

### Log Path

In [13]:
log_path = './mAP_&_pr_curves/classify_&_detect/'

# BED manual optim DFire

## Score Thres = 0.2

In [14]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = bed_manual_optim_name,
    score_thres = 0.2
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [02:01<00:00,  1.96s/it]


In [15]:
score_02_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  2.73it/s]


In [16]:
print_metrics(score_02_metrics)

mAP @0.50   0.3542
Average Precision
- Smoke     0.4485
- Fire      0.2600
Class Precision
- Smoke     0.6325
- Fire      0.4775
Class Recall
- Smoke     0.5511
- Fire      0.3725
Class F1-Score
- Smoke     0.5890
- Fire      0.4185


## Score Thres = 0.001

In [17]:
pred_boxes, true_boxes = aladdin_get_bboxes(
    detection_loader,
    detector_model_name = bed_manual_optim_name,
    score_thres = 0.001
)

Get Boxes: 100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [04:42<00:00,  4.55s/it]


In [18]:
score_0001_metrics = aladdin_metrics.mAP(
    log_path=log_path + 'score_thres_2e-1/',
    pred_boxes=pred_boxes,
    true_boxes=true_boxes)

mAP:@.5: 100%|███████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.47s/it]


In [19]:
print_metrics(score_0001_metrics)

mAP @0.50   0.3912
Average Precision
- Smoke     0.5033
- Fire      0.2792
Class Precision
- Smoke     0.0140
- Fire      0.0777
Class Recall
- Smoke     0.7200
- Fire      0.4333
Class F1-Score
- Smoke     0.0275
- Fire      0.1318
